In [ ]:
# Pandas

import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['mpg', 'cyl', 'disp', 'hp', 'weight', 'acc', 'year', 'origin'] 

df = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
df

In [ ]:
df['mpg']

In [ ]:
df['mpg'].values[:5]

In [ ]:
df[['mpg', 'hp']]

In [ ]:
df[['mpg', 'hp']].values[:5]

In [ ]:
df.iloc[:5]

In [ ]:
cond = (df['year'] == 70)
df[cond]

In [ ]:
# Train-Validation-Test Split: Shuffling

df['year'].values[:50]

In [ ]:
shuffled = df.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

trainval, test = train_test_split(shuffled, test_size=0.16, shuffle=False)
train, val = train_test_split(trainval, test_size=0.2, shuffle=False)

In [ ]:
# Cleaning Data

is_missing_attr = train.isna()
n_missing_attr = is_missing_attr.sum(axis=1)
train[n_missing_attr > 0]

In [ ]:
train.dropna(inplace=True)
train

In [ ]:
val.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
# Continuous Attributes

cont_attr = ['mpg', 'disp', 'hp', 'weight', 'acc']

In [ ]:
train_features = train[cont_attr[1:]]
train_features.hist()

In [ ]:
train_means = train_features.mean()
train_standard_deviations = train_features.std()
train_means, train_standard_deviations

In [ ]:
train_standardized_features = (train_features - train_means)/train_standard_deviations
train_standardized_features.mean(), train_standardized_features.std()

In [ ]:
train_standardized_features.hist()

In [ ]:
val_features = val[cont_attr[1:]]
val_standardized_features = (val_features - train_means)/train_standard_deviations
val_standardized_features.mean(), val_standardized_features.std()

In [ ]:
test_features = test[cont_attr[1:]]
test_standardized_features = (test_features - train_means)/train_standard_deviations

In [ ]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
scaler.fit(train_features.values)

In [ ]:
scaler.mean_, scaler.var_

In [ ]:
standardized_data = {}
standardized_data['train'] = scaler.transform(train_features)
standardized_data['val'] = scaler.transform(val_features)
standardized_data['test'] = scaler.transform(test_features)

In [ ]:
from sklearn.preprocessing import StandardScaler 

def standardize(df, cont_attr, scaler=None):
    cont_X = df[cont_attr].values
    if scaler is None:
        scaler = StandardScaler()
    scaler.fit(cont_X)
    cont_X = scaler.transform(cont_X)
    cont_X = torch.as_tensor(cont_X, dtype=torch.float32)
    return cont_X, scaler

In [ ]:
standardized_data = {}
# The training set is used to fit a scaler
standardized_data['train'], scaler = standardize(train_features, cont_attr[1:])
# The scaler is used as argument to the other datasets
standardized_data['val'], _ = standardize(val_features, cont_attr[1:], scaler)
standardized_data['test'], _ = standardize(test_features, cont_attr[1:], scaler)

In [ ]:
# Discrete and Categorical Attributes

cyls = sorted(train['cyl'].unique())
cyls

In [ ]:
cyls_map = dict((v, i) for i, v in enumerate(cyls))
cyls_map

In [ ]:
n_dim = 8
lookup_table = torch.randn((len(cyls), n_dim))
lookup_table

In [ ]:
idx = cyls_map[6]
lookup_table[idx]

In [ ]:
import torch.nn as nn 

emb_table = nn.Embedding(len(cyls), n_dim)

In [ ]:
idx = cyls_map[6]
emb_table(torch.as_tensor([idx]))

In [ ]:
ohe_table = torch.eye(len(cyls))
ohe_table

In [ ]:
idx = cyls_map[6]
ohe_table[idx]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
disc_attr = ['cyl', 'year', 'origin'] 

encoder = OrdinalEncoder()
encoder.fit(train[disc_attr])

In [ ]:
encoder.categories_

In [ ]:
train_cat_features = encoder.transform(train[disc_attr])
train_cat_features[:5]

In [ ]:
train[disc_attr].iloc[0]

In [ ]:
def encode(df, cat_attr, encoder=None):
    cat_X = df[cat_attr].values
    if encoder is None:
        encoder = OrdinalEncoder()
        encoder.fit(cat_X)
    cat_X = encoder.transform(cat_X)
    cat_X = torch.as_tensor(cat_X, dtype=torch.int)
    return cat_X, encoder

In [ ]:
cat_data = {}
cat_data['train'], encoder = encode(train, disc_attr)
cat_data['val'], _ = encode(val, disc_attr, encoder)
cat_data['test'], _ = encode(test, disc_attr, encoder)

In [ ]:
emb_table(cat_data['train'][:, 0]) # cylinders is the first (zero) column